In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
load_dotenv('.env')

True

In [2]:
date = datetime.today() - timedelta(weeks=4)
date = date.strftime('%Y-%m-%d')
date

'2024-12-13'

In [3]:
from src.extract import extract_news

news_url = os.getenv('NEWSAPI_EP')
news_apikey = os.getenv('APIKEY_NEWSAPI')
news_url = news_url.replace('[apiKey]', news_apikey)
news_url = news_url.replace('[date]', date)

print(news_url)

https://newsapi.org/v2/everything?q=[coin_symbol]&apiKey=67a8021b9bf944228789c68ea30931cd&from=2024-12-13&sortBy=popularity


In [4]:
tgt_dir = os.getenv('EXTRACTED_DIR')
extract_news(news_url,'bitcoin',tgt_dir, 'news')

Successfully saved to data/extracted_data/news/bitcoin.json.



In [31]:
coinlists = pd.read_json('./data/extracted_data/coinslist/coinslist.json')
coin_symbols = coinlists['symbol']
coin_symbols

0         zoc
1         zcn
2        0dog
3         0kn
4         ome
         ... 
16555     zyn
16556     zyn
16557     zyr
16558     zzz
16559       ᚠ
Name: symbol, Length: 16560, dtype: object